In [2]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [1]:
OPENAI_API_KEY = 'xxx'

In [19]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [4]:
from langchain_community.document_loaders import WebBaseLoader

In [5]:
url = "https://de.wikipedia.org/wiki/Vela-Supernova"

In [7]:
loader = WebBaseLoader(url)


In [8]:
document = loader.load()
document

[Document(page_content="\n\n\n\nVela-Supernova – Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nVela-Supernova\n\naus Wikipedia, der freien Enzyklopädie\n\n\n\nZur Navigation springen\nZur Suche springen\nDatenbanklinks zu Vela-Supernova\n\n\nSupernova\n\n\nVela-Supernova\n\n\nAufnahme des durch die Vela-Supernova entstandenen Pulsars (punktförmig weiß, mittig) und des ihn umgebenden Nebels im Röntgenbereich mithilfe des Satellitenobservatoriums ROSAT.  Die darin scheinbar überlagerte, jüngere Supernova Puppis A zeichnet sich rechts oben hellblau ab.\n\n\nAufnahme des durch die Vela-Supernova entstandenen Pulsars (punktförmig weiß, mittig) und des ihn umgebenden Nebels im Röntgenbereich mithilfe des Satellitenobservatoriums ROSAT.\nDie darin scheinbar überlagerte, jüngere Supernova Puppis A zeichnet sich rechts oben hellblau ab.\n\n\n\nSternbild\n\nSegel des Schiffs\n\n\nPositionÄquinoktium: J2000.0\n\n\nRektaszension\n\n08h 35m 20,

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter()

In [12]:
document_chunks = text_splitter.split_documents(document)
document_chunks

[Document(page_content='Vela-Supernova – Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nVela-Supernova\n\naus Wikipedia, der freien Enzyklopädie\n\n\n\nZur Navigation springen\nZur Suche springen\nDatenbanklinks zu Vela-Supernova\n\n\nSupernova\n\n\nVela-Supernova\n\n\nAufnahme des durch die Vela-Supernova entstandenen Pulsars (punktförmig weiß, mittig) und des ihn umgebenden Nebels im Röntgenbereich mithilfe des Satellitenobservatoriums ROSAT.  Die darin scheinbar überlagerte, jüngere Supernova Puppis A zeichnet sich rechts oben hellblau ab.\n\n\nAufnahme des durch die Vela-Supernova entstandenen Pulsars (punktförmig weiß, mittig) und des ihn umgebenden Nebels im Röntgenbereich mithilfe des Satellitenobservatoriums ROSAT.\nDie darin scheinbar überlagerte, jüngere Supernova Puppis A zeichnet sich rechts oben hellblau ab.\n\n\n\nSternbild\n\nSegel des Schiffs\n\n\nPositionÄquinoktium: J2000.0\n\n\nRektaszension\n\n08h 35m 20,66s[1]\n

In [13]:
from langchain_community.vectorstores import Chroma

In [17]:
vector_store = Chroma.from_documents(document_chunks, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

In [18]:
vector_store

In [20]:
retriever = vector_store.as_retriever()

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [22]:
prompt = ChatPromptTemplate.from_messages([
      # MessagesPlaceholder(variable_name="chat_history"),
      ("user", "{input}"),
      ("user", "Given the above question, generate a search query to look up in order to get information relevant to the question")
    ])

In [31]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

In [24]:
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [27]:
retriever_chain.invoke({
    "input": 'Wer hat die Supernova beobachtet?'
})

[Document(page_content='2 Beobachtbarkeit\n3 Rezeption\n\n3.1 Kunst\n3.2 Populärwissenschaftliche Berichte – Hypothesen über Effekte und Artefakte\n\n\n4 Literatur\n5 Weblinks\n6 Einzelnachweise\n\n\nEntdeckung und Erforschung[Bearbeiten | Quelltext bearbeiten]\nSupernovaüberrest[Bearbeiten | Quelltext bearbeiten]\nHochaufgelöste Aufnahme einiger Filamente mithilfe von Spektralfilter für SII (grün), OIII (blau), Hα und NII (rot) des Víctor M. Blanco Telescope zeigt in Falschfarben Verteilung und Stoßfronten der ionisierten Gase.\nIm Sternbild Vela fand bereits im Jahr 1835 John Herschel bei seiner Durchmusterung des Südhimmels einen filamentartigen Nebel, den Bleistiftnebel. In dessen Umgebung entdeckten 1926 Philibert Jacques Melotte und Knut Lundmark auf den empfindlicheren Fotografien einer Anfang des 20. Jahrhunderts durchgeführten Himmelsdurchmusterung[7] eine Vielzahl Nebelstreifen, die zusammen einen etwa ovalen, am Firmament scheinbar 7 Quadratgrad großen Nebel bildeten.[8] Bei

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [29]:
answer_prompt = ChatPromptTemplate.from_messages([
    ("system","Answer the user's question based on the below context:\n\n{context}"),
    # MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

stuff_documents_chain = create_stuff_documents_chain(llm, answer_prompt)

In [33]:
conversation_rag_chain = create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [39]:
response = conversation_rag_chain.invoke({
    "input": 'Wo hat die Supernova stattgefunden?'
})

In [40]:
response['answer']

'Die Supernova, die als Vela-Supernova bekannt ist, ereignete sich im südlichen Sternbild Segel des Schiffs, lateinisch Vela. Es handelt sich um eine nahegelegene Sternenexplosion, die sich vor einigen tausend Jahren etwa 290 Parsec von der Erde entfernt ereignete.'